In [1]:
from cProfile import run
from dask.distributed import Client, LocalCluster
import time
import json
import csv
from datetime import datetime
import dask.dataframe as dd

def PA1(user_reviews_csv,products_csv):
    start = time.time()
    client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='18GB')
    client = client.restart()
    print(client)
    
    # Q1
    df_reviews = dd.read_csv(user_reviews_csv)
    df_products = dd.read_csv(products_csv)

    q1_reviews = df_reviews.isna().mean()
    q1_products = df_products.isna().mean()
    
    #Q2
    asin_rating = df_reviews.groupby('asin').aggregate({
        'overall' : 'mean'
    })
    asin_rating.columns = ['avg_rating']
    asin_price = df_products.groupby('asin').aggregate({
        'price' : 'sum'
    })
    #print(asin_rating.head())
    #print(asin_price.head())
    asin_merged = dd.merge(asin_rating, asin_price, on='asin').dropna()
    #print(asin_merged.head())
    q2 = asin_merged['price'].corr(asin_merged['avg_rating'])
    # This will make more sense on larger datasets.
    
    #Q3
    q3 = [
        asin_price['price'].mean(),
        asin_price['price'].std(),
        asin_price['price'].quantile(0.5),
        asin_price['price'].min(),
        asin_price['price'].max()
    ]
    
    #Q4
    
    def extract_super_category(row):
        row = str(row)
        if len(row) <= 4:
            return None
        reader = csv.reader([row[2:-2]], delimiter=',', quotechar='\'')
        categories = next(reader)
        if categories:
            return categories[0]
    df_products['super_category'] = df_products['categories'].apply(extract_super_category)
    
    #print(df_products['super_category'].head())
    q4 = df_products.groupby('super_category').aggregate({
        'asin' : 'count'
    })
    
    #Q5
    q5 = None
    
    
    #Q6
    #Parse JSON, then same as #5
    q6 = None
    
    
    end = time.time()
    runtime = end-start
    return runtime

    # Write your results to "results_PA1.json" here
    with open('OutputSchema_PA1.json','r') as json_file:
        data = json.load(json_file)
        print(data)

        data['q1']['products'] = json.loads(q1_reviews.to_json())
        data['q1']['reviews'] = json.loads(q1_products.to_json())
        data['q2'] = q2
        data['q3'] = json.loads(q3.to_json())
        data['q4'] = json.loads(q4.to_json())
        data['q5'] = q5
        data['q6'] = q6
    
    # print(data)
    with open('results_PA1.json', 'w') as outfile: json.dump(data, outfile)

    
    return runtime

PA1("small_user_reviews_Release.csv", "small_products_Release.csv")
#PA1("user_reviews_Release.csv", "products_Release.csv")

<Client: 'tcp://127.0.0.1:38803' processes=1 threads=4, memory=16.78 GB>


/home/ubuntu/dask_env/lib/python3.6/site-packages/dask/dataframe/multi.py:1193: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  "Concatenating dataframes with unknown divisions.\n"
/home/ubuntu/dask_env/lib/python3.6/site-packages/dask/dataframe/core.py:3370: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('categories', 'object'))

  warnings.warn(meta_warning(meta))


1.6312506198883057